In [1]:
import warnings
# Suppress all warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
import os
from dotenv import find_dotenv
from dotenv import load_dotenv

sys.path.append('../..')
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
# !pip install neo4j

In [4]:

from neo4j import GraphDatabase

user_name = os.environ['user_name']
password = os.environ['password']
url = os.environ['url']

driver = GraphDatabase.driver(url,auth=(user_name, password))

In [5]:

def add_friend(tx, name, friend_name):
    tx.run("MERGE (a:Person {name: $name}) "
           "MERGE (a)-[:KNOWS]->(friend:Person {name: $friend_name})",
            name=name, friend_name=friend_name)


In [6]:

def print_friends(tx, name):
    for record in tx.run("MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name "
                          "RETURN friend.name ORDER BY friend.name", name=name):
        print(record["friend.name"])


In [7]:

with driver.session() as session:
    session.write_transaction(add_friend, "Arthur", "Guinevere")
    session.write_transaction(add_friend, "Arthur", "Lancelot")
    session.write_transaction(add_friend, "Arthur", "Merlin")
    session.read_transaction(print_friends, "Arthur")


Guinevere
Lancelot
Merlin


In [8]:
# !pip install py2neo

In [9]:

from py2neo import Graph, Node, Relationship


In [10]:

g = Graph(url, auth=(user_name, password))

In [11]:

tx = g.begin()
a = Node("Person", name="Alice")
tx.create(a)
b = Node("Person", name="Bob")

ab = Relationship(a, "KNOWS", b)

tx.create(ab)
tx.commit()


In [12]:
from py2neo import Graph, Node, Relationship, Subgraph


g = Graph(url, auth=(user_name, password))


In [13]:

g.delete_all()


In [14]:

tx = g.begin()
names = ['John', 'Sally', 'Steve', 'Mike', 'Liz', 'Shawn']
for p_name in names:
    p_node = Node("Person", name=p_name)
    tx.create(p_node)
tx.commit()


In [15]:

tx = g.begin()
locations = [{'city':'Miami', 'state':'FL'}, {'city':'Boston', 'state':'MA'}, {'city':'Lynn', 'state':'MA'}, 
             {'city':'Portland', 'state':'ME'}, {'city':'San Francisco', 'state':'CA'}]
for location in locations:
    l_node = Node("Location", city=location['city'], state=location['state'])
    tx.create(l_node)
tx.commit()


In [16]:

tx = g.begin()
a = tx.graph.nodes.match("Person", name='Liz').first()
b = tx.graph.nodes.match("Person", name='Mike').first()
ab = Relationship(a, "FRIENDS", b)
tx.create(ab)
tx.commit()


In [17]:

tx = g.begin()
a = tx.graph.nodes.match("Person", name='Shawn').first()
b = tx.graph.nodes.match("Person", name='Sally').first()
ab = Relationship(a, "FRIENDS", b, since=2001)
tx.create(ab)
tx.commit()


In [18]:

tx = g.begin()
a = tx.graph.nodes.match("Person", name='Shawn').first()
b = tx.graph.nodes.match("Person", name='John').first()
c = tx.graph.nodes.match("Person", name='Mike').first()
d = tx.graph.nodes.match("Person", name='Sally').first()
e = tx.graph.nodes.match("Person", name='Steve').first()
f = tx.graph.nodes.match("Person", name='Liz').first()
ab = Relationship(a, "FRIENDS", b, since=2012)
ca = Relationship(c, "FRIENDS", a, since=2006)
de = Relationship(d, "FRIENDS", e, since=2006)
fb = Relationship(f, "MARRIED", b, since=1998)
A = Subgraph(relationships=[ab, ca, de, fb])   
tx.create(A)
tx.commit()


In [19]:

tx = g.begin()
p_John = tx.graph.nodes.match("Person", name='John').first()
p_Liz = tx.graph.nodes.match("Person", name='Liz').first()
p_Mike = tx.graph.nodes.match("Person", name='Mike').first()
p_Shawn = tx.graph.nodes.match("Person", name='Shawn').first()
p_Steve = tx.graph.nodes.match("Person", name='Steve').first()

l_Boston = tx.graph.nodes.match("Location", city='Boston').first()
l_Francisco = tx.graph.nodes.match("Location", city='San Francisco').first()
l_Miami = tx.graph.nodes.match("Location", city='Miami').first()
l_Lynn = tx.graph.nodes.match("Location", city='Lynn').first()

p_l_1 =  Relationship(p_John, "BORN_IN", l_Boston, year=1978)
p_l_2 =  Relationship(p_Liz, "BORN_IN", l_Boston, year=1981)
p_l_3 =  Relationship(p_Mike, "BORN_IN", l_Francisco, year=1960)
p_l_4 =  Relationship(p_Shawn, "BORN_IN", l_Miami, year=1960)
p_l_5 =  Relationship(p_Steve, "BORN_IN", l_Lynn, year=1970)
A = Subgraph(relationships=[p_l_1, p_l_2, p_l_3, p_l_4, p_l_5])   
tx.create(A)
tx.commit()


In [20]:

b = g.nodes.match("Location", city="Boston").first()
r1 = g.relationships.match({b}, 'BORN_IN').all()
print(r1)


[BORN_IN(Node('Person', name='John'), Node('Location', city='Boston', state='MA'), year=1978), BORN_IN(Node('Person', name='Liz'), Node('Location', city='Boston', state='MA'), year=1981)]
